In [1]:
'''
RNN for MNIST digits classification
97.7% test accuracy in 20epochs
75K adjustable parameters
input -> 256 -> 10

https://github.com/PacktPublishing/Advanced-Deep-Learning-with-Keras
'''

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, SimpleRNN, LSTM, Input, Bidirectional
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import mnist



In [2]:
!python --version
!pip list | grep tensorflow

Python 3.6.9
tensorflow                    2.4.0          
tensorflow-addons             0.8.3          
tensorflow-datasets           4.0.1          
tensorflow-estimator          2.4.0          
tensorflow-gcs-config         2.4.0          
tensorflow-hub                0.10.0         
tensorflow-metadata           0.26.0         
tensorflow-privacy            0.2.2          
tensorflow-probability        0.11.0         


## Clean Data

In [3]:
# load mnist dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# compute the number of labels
num_labels = len(np.unique(y_train))

# convert to one-hot vector
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# resize and normalize
#image_size = x_train.shape[1]
#print(x_train.shape)

x_train = np.reshape(x_train,[x_train.shape[0], -1, 1])
x_test = np.reshape(x_test,[x_test.shape[0], -1, 1])

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

print("x_train", x_train.shape)
print("y_train", y_train.shape)

print("x_test", x_test.shape)
print("y_test", y_test.shape)

11493376/11490434 [==============================] - 0s 0us/step
x_train (60000, 784, 1)
y_train (60000, 10)
x_test (10000, 784, 1)
y_test (10000, 10)


## Setup Model

In [4]:
# network parameters
input_shape = (x_train.shape[1:])
batch_size = 128
units = 100
dropout = 0.00

# model is RNN with 256 units, input is 28-dim vector 28 timesteps
model = Sequential()
model.add(Input(input_shape, name="input"))
model.add(Bidirectional(LSTM(units=units,
                    dropout=dropout, name="ltsm" )))
                    #input_shape=input_shape))
model.add(Dense(num_labels, name="dense"))
model.add(Activation('softmax', name="activate"))
model.summary()
print("units", units)
print("input_shape", input_shape)
print("num_labels", num_labels)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 200)               81600     
_________________________________________________________________
dense (Dense)                (None, 10)                2010      
_________________________________________________________________
activate (Activation)        (None, 10)                0         
Total params: 83,610
Trainable params: 83,610
Non-trainable params: 0
_________________________________________________________________
units 100
input_shape (784, 1)
num_labels 10


## Model compile and fit (train)

In [ ]:
# enable this if pydot can be installed
# pip install pydot
#plot_model(model, to_file='rnn-mnist.png', show_shapes=True)

# loss function for one-hot vector
# use of sgd optimizer
# accuracy is good metric for classification tasks

from keras.optimizers import RMSprop

learning_rate = 1e-3
rmsprop = RMSprop(lr=learning_rate)

model.compile(loss='categorical_crossentropy',
              optimizer= rmsprop,
              metrics=['accuracy'])

#model.compile(loss='categorical_crossentropy',
#              optimizer='sgd',
#              metrics=['accuracy'])
# train the network

model.fit(x_train, y_train, 
          epochs=200, 
          batch_size=batch_size,
          validation_data=(x_test, y_test))



Epoch 1/200
469/469 [==============================] - 45s 83ms/step - loss: 2.2912 - accuracy: 0.1301 - val_loss: 2.3129 - val_accuracy: 0.1009
Epoch 2/200
469/469 [==============================] - 38s 81ms/step - loss: 2.3070 - accuracy: 0.1082 - val_loss: 2.3023 - val_accuracy: 0.1135
Epoch 3/200
469/469 [==============================] - 38s 81ms/step - loss: 2.3015 - accuracy: 0.1080 - val_loss: 2.1965 - val_accuracy: 0.1814
Epoch 4/200
469/469 [==============================] - 38s 82ms/step - loss: 2.2080 - accuracy: 0.1786 - val_loss: 2.2129 - val_accuracy: 0.1830
Epoch 5/200
469/469 [==============================] - 38s 82ms/step - loss: 2.0843 - accuracy: 0.2378 - val_loss: 2.2500 - val_accuracy: 0.1801
Epoch 6/200
469/469 [==============================] - 39s 82ms/step - loss: 2.0671 - accuracy: 0.2256 - val_loss: 2.3244 - val_accuracy: 0.1135
Epoch 7/200
469/469 [==============================] - 38s 82ms/step - loss: 2.2765 - accuracy: 0.1388 - val_loss: 2.5537 - val_ac

## Model pred

In [ ]:
_, acc = model.evaluate(x_test,
                        y_test,
                        batch_size=batch_size,
                        verbose=0)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))